In [3]:
import os
import sys
import numpy as np

from mrcnn.model import MaskRCNN, load_image_gt, mold_image
from mrcnn.utils import compute_ap
from tdmms.tdmcoco import CocoConfig
from bep.utils import load_train_val_datasets

ROOT_DIR = os.path.abspath("../")
print('Root directory:',ROOT_DIR)
sys.path.append(ROOT_DIR)

Root directory: c:\Users\abell\Documents\aa_delft\J4\BEP\ai


In [4]:
class InferenceConfig(CocoConfig):
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    NAME = 'inference'

DEFAULT_LOGS_DIR = os.path.join(ROOT_DIR, 'logs', 'training')

if not os.path.exists(DEFAULT_LOGS_DIR):
    os.makedirs(DEFAULT_LOGS_DIR)
    print(f"Folder '{DEFAULT_LOGS_DIR}' created.")

In [ ]:
def calculate_mean_average_precision(dataset, inference_model):
    mAPs = []
    dataset_limit = len(dataset.image_ids)
    dataset_image_ids = dataset.image_ids.copy()

    # Use a random subset of the data when a limit is defined
    np.random.shuffle(dataset_image_ids)

    for image_id in dataset_image_ids[:dataset_limit]:
        image, image_meta, gt_class_id, gt_bbox, gt_mask = load_image_gt(dataset, inference_model.config, image_id)
        molded_images = np.expand_dims(mold_image(image, inference_model.config), 0)
        results = inference_model.detect(molded_images, verbose=0)
        r = results[0]
        
        # Compute mAP - VOC uses IoU 0.5
        AP, _, _, _ = compute_ap(
            gt_bbox,
            gt_class_id,
            gt_mask,
            r["rois"],
            r["class_ids"],
            r["scores"],
            r['masks']
        )
        mAPs.append(AP)

    return np.array(mAPs)

In [6]:
_, _, test = load_train_val_datasets(ROOT_DIR)

In [9]:
config = InferenceConfig()
inference_model = MaskRCNN(
    mode="inference",
    config=config,
    model_dir=DEFAULT_LOGS_DIR
)

inference_model.load_weights(os.path.join(ROOT_DIR, 'weights', '20241123-173838_nbse2_wte2_4_True_89_2__0113.h5'), by_name=True)

In [10]:
mAPs = calculate_mean_average_precision(test, inference_model)
print('mAPs: {}'.format(mAPs))
mAP = np.mean(mAPs)
print('mean mAP: {}'.format(mAP))

c:\Users\abell\Documents\aa_delft\J4\BEP\ai\env_tf24\lib\site-packages\tensorflow\python\keras\engine\training.py:2325: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


ValueError: shapes (19,1048576) and (3136,10) not aligned: 1048576 (dim 1) != 3136 (dim 0)